# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [22]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [10]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [19]:
# system prompt

system_prompt = """
You are an assistant that answers questions concisely with technical details. The answer should be as descriptive as possible,
using language that can be understood by a 5 year old.
"""

# here is the question; type over this to ask something new

user_prompt = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [18]:
# Get gpt-4o-mini to answer, with streaming

def get_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

get_answer(user_prompt)

Alright! Imagine you have a magical treasure chest (that’s your code!) filled with sparkly books, and each book has a tag telling you who wrote it—the author! Now, let’s break down what’s happening:

1. **Books Galore**: Think of `books` as a big shelf where all your storybooks are sitting in a neat line. Some books say who the author is, and some books are a bit shy and just sit there without telling you.

2. **Searching for Authors**: The part `book.get("author")` is like asking each book, "Hey, who wrote you?" If the book says "Me! I'm written by Captain Awesome!" then yay! You found an author! If the book goes “Umm... I forgot!”, you just move on to the next one.

3. **Creating a List**: The fancy `{ ... for book in books if book.get("author")}` part means you're gathering all the names of authors from the books that told you who they are. It's like putting a name sticker on all the books that have an author! 

4. **Sparkling Unique Names**: The curly braces `{}` mean you’re creating something special—a *set*! This is like a group of friends where nobody can have the same name, so if two books are written by Captain Awesome, he only gets one sticker!

5. **The Magic of Yielding**: Finally, the `yield from` part is like saying, "Okay, magic treasure chest, please start handing me those names one by one!" It allows you to take each author’s name out, one at a time, as if you’re pulling out candy from a piñata—sweet and delightful!

So, to sum it up with a giggle: This code is like a magical helper that goes through your book shelf, gathers all the brave authors who shouted their names, puts them in a special list without repeats, and then hands them out to you one by one like a candy parade! 🍭✨

In [24]:
# Get Llama 3.2 to answer

# Using ollama python package

def get_answer(question):
    ollama_response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    # Printing out each piece of the generated response while preserving order
    for chunk in ollama_response:
        print(chunk['message']['content'], end='', flush=True)


get_answer(user_prompt)


This line of code is doing something really cool with a bunch of books!

Let's break it down:

- `books` is like a big box full of books.
- Each book is like a small container that has some information inside, like the author's name.

Here's what this line does:

1. It takes all the books in the box (`for book in books`).
2. For each book, it checks if the book has an "author" inside (like a small treasure chest with a key: `"author"`).
3. If a book does have an author (like finding a treasure), it gets the author's name from that book.
4. It then makes the author's name available to someone else who is waiting for more information, like a helper.

The magic part is `yield from`. Imagine you're on a treasure hunt with your friend. You find a clue and say "Look! This is a map!" Then, your friend can look at the map and see where the next clue is. That's kind of what `yield from` does: it says "Hey, I found this information, here's more info about where to go next"!

In code terms, when 